<a href="https://colab.research.google.com/github/Me-Vish/TASK1_AI_WEATHER_FORECASTING/blob/main/DAY12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===============================
# 1. Imports
# ===============================
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dot, Activation, Concatenate
import numpy as np

# ===============================
# 2. Dataset (Improved & Balanced)
# ===============================
english_sentences = [
    "i like coffee",
    "she likes tea",
    "he eats fruits",
    "we go to college",
    "they play cricket",
    "i read newspapers",
    "you drink milk",
    "she watches tv",
    "he listens to music",
    "we learn coding",
    "they visit temples",
    "i cook food",
    "you help me",
    "she teaches students",
    "he drives a car",
    "we clean the house",
    "they buy vegetables",
    "i sleep early",
    "you speak english",
    "she loves her mother"
]

tamil_sentences = [
    " எனக்கு காபி பிடிக்கும் ",
    " அவளுக்கு தேநீர் பிடிக்கும் ",
    " அவன் பழங்கள் சாப்பிடுகிறான் ",
    " நாங்கள் கல்லூரிக்கு செல்கிறோம் ",
    " அவர்கள் கிரிக்கெட் விளையாடுகிறார்கள் ",
    " நான் செய்தித்தாள்கள் படிக்கிறேன் ",
    " நீங்கள் பால் குடிக்கிறீர்கள் ",
    " அவள் தொலைக்காட்சி பார்க்கிறாள் ",
    " அவன் இசை கேட்கிறான் ",
    " நாங்கள் கோடிங் கற்றுக்கொள்கிறோம் ",
    " அவர்கள் கோவில்களுக்கு செல்கிறார்கள் ",
    " நான் உணவு சமைக்கிறேன் ",
    " நீங்கள் எனக்கு உதவுகிறீர்கள் ",
    " அவள் மாணவர்களுக்கு பாடம் கற்பிக்கிறாள் ",
    " அவன் கார் ஓட்டுகிறான் ",
    " நாங்கள் வீட்டை சுத்தம் செய்கிறோம் ",
    " அவர்கள் காய்கறிகள் வாங்குகிறார்கள் ",
    " நான் சீக்கிரம் தூங்குகிறேன் ",
    " நீங்கள் ஆங்கிலம் பேசுகிறீர்கள் ",
    " அவள் தன் அம்மாவை நேசிக்கிறாள் "
]
# ===============================
# 3. Tokenization
# ===============================
src_tokenizer = Tokenizer(oov_token="")
src_tokenizer.fit_on_texts(english_sentences)
src_seq = src_tokenizer.texts_to_sequences(english_sentences)

tgt_tokenizer = Tokenizer(filters='', oov_token="")
tgt_tokenizer.fit_on_texts(tamil_sentences)
tgt_seq = tgt_tokenizer.texts_to_sequences(tamil_sentences)

src_vocab_size = len(src_tokenizer.word_index) + 1
tgt_vocab_size = len(tgt_tokenizer.word_index) + 1

# ===============================
# 4. Padding
# ===============================
max_src_len = max(len(s) for s in src_seq)
max_tgt_len = max(len(s) for s in tgt_seq)

encoder_input = pad_sequences(src_seq, maxlen=max_src_len, padding='post')

decoder_input = pad_sequences(
    [s[:-1] for s in tgt_seq],
    maxlen=max_tgt_len-1,
    padding='post'
)

decoder_target = pad_sequences(
    [s[1:] for s in tgt_seq],
    maxlen=max_tgt_len-1,
    padding='post'
)

# ===============================
# 5. Model Parameters (Small = Stable)
# ===============================
embedding_dim = 32
latent_dim = 64

# ===============================
# 6. Encoder
# ===============================
encoder_inputs = Input(shape=(max_src_len,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)

encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# ===============================
# 7. Decoder
# ===============================
decoder_inputs = Input(shape=(max_tgt_len-1,))
dec_emb = Embedding(tgt_vocab_size, embedding_dim)(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# ===============================
# 8. Attention
# ===============================
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined = Concatenate(axis=-1)([context, decoder_outputs])

# ===============================
# 9. Output Layer
# ===============================
output = Dense(tgt_vocab_size, activation='softmax')(decoder_combined)

# ===============================
# 10. Compile Model
# ===============================
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ===============================
# 11. Train
# ===============================
model.fit(
    [encoder_input, decoder_input],
    np.expand_dims(decoder_target, -1),
    batch_size=4,
    epochs=400,
    shuffle=False,
    verbose=1
)

# ===============================
# 12. Inference (Translate Function)
# ===============================
reverse_tgt_index = {v: k for k, v in tgt_tokenizer.word_index.items()}

def translate(sentence):
    seq = src_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_src_len, padding='post')

    target_seq = np.zeros((1, max_tgt_len-1))
    target_seq[0, 0] = tgt_tokenizer.word_index['']

    decoded = []

    for i in range(max_tgt_len-1):
        preds = model.predict([seq, target_seq], verbose=0)
        word_id = np.argmax(preds[0, i])
        word = reverse_tgt_index.get(word_id, '')

        if word == '' or word == '':
            break

        decoded.append(word)
        target_seq[0, i+1] = word_id

    return ' '.join(decoded)

# ===============================
# 13. Test
# ===============================
print("English :", "i like coffee")
print("Tamil   :", translate("i like coffee"))

print("English :", "we go to college")
print("Tamil   :", translate("we go to college"))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4, 32)     │      1,664 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 3, 32)     │      1,632 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 4, 64),   │     24,832 │ embedding[0][0]   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 3, 64),   │     24,832 │ embedding_1[0][0… │
│                     │ (None, 64),       │            │ lstm[0][1],       │
│                     │ (None, 64)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 3, 4)      │          0 │ lstm_1[0][0],     │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 3, 4)      │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 3, 64)     │          0 │ activation[0][0], │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3, 128)    │          0 │ dot_1[0][0],      │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3, 51)     │      6,579 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 59,539 (232.57 KB)

 Trainable params: 59,539 (232.57 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.1306 - loss: 3.9289   
Epoch 2/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3097 - loss: 3.9067 
Epoch 3/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3097 - loss: 3.8831
Epoch 4/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3097 - loss: 3.8511
Epoch 5/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3097 - loss: 3.8029
Epoch 6/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3097 - loss: 3.7237
Epoch 7/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3097 - loss: 3.5821
Epoch 8/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3097 - loss: 3.3183
Epoch 9/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3097 - loss: 2.9625
Epoch 10/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3097 - loss: 2.9064
Epoch 11/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3097 - loss: 2.8240
Epoch 12/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3097 